<div dir="rtl" style="text-align: right;">
<h1>چیدمان حافظه</h1>




<p><a href="https://docs.scipy.org/doc/numpy/reference/arrays.ndarray.html">مستندات NumPy</a> کلاس <code>ndarray</code> را بسیار روشن تعریف می‌کند:</p>
<blockquote>
<p>یک نمونه از کلاس <code>ndarray</code> شامل یک بخش یک‌بعدی پیوسته از حافظه کامپیوتری (متعلق به آرایه، یا توسط شیء دیگری) است، همراه با یک طرح فهرست‌بندی که N عدد صحیح را به مکان یک مورد در بلوک ارجاع می‌دهد.</p>
</blockquote>

<p>به عبارت دیگر، یک آرایه عمدتاً یک بلوک پیوسته حافظه است که می‌توان به قسمت‌های آن با استفاده از یک طرح فهرست‌بندی دسترسی پیدا کرد. چنین طرح فهرست‌بندی به نوبه خود توسط یک شکل
shape
 و نوع داده
 data type
  تعریف می‌شود و این دقیقاً همان چیزی است که زمانی که شما یک آرایه جدید تعریف می‌کنید، نیاز دارید.</p>
</div>


In [2]:
import numpy as np
Z = np.arange(9).reshape(3,3).astype(np.int16)
print(Z)
print(Z.itemsize)# اندازه Z را به بایت برمی‌گرداند
print(Z.shape)# ابعاد x و y آرایه Z را برمی‌گرداند
print(Z.ndim)# بعد در Z، یعنی ۲ در این مورد چون آرایه دوبعدی است

[[0 1 2]
 [3 4 5]
 [6 7 8]]
2
(3, 3)
2


<div dir="rtl" style="text-align: right;">
<p>در اینجا، ما می‌دانیم که اندازه هر آیتم ۲ بایت است (int16)، شکل آن (3,3) و تعداد بعدها ۲ است.</p>
<blockquote>
برای محاسبه بُعد می‌توانیم از <code>len(Z.shape)</code> نیز استفاده کنیم.
</blockquote>
<p>علاوه بر این، ما می‌توانیم <a href="https://docs.scipy.org/doc/numpy/reference/generated/numpy.ndarray.strides.html#numpy.ndarray.strides">قدم‌های</a> آرایه را که تعداد بایت‌هایی را تعریف می‌کنند که باید در هر بُعد هنگام عبور از آرایه گام برداشت، نتیجه‌گیری کنیم.</p>



</div>


In [3]:
import numpy as np
Z = np.arange(9).reshape(3,3).astype(np.int16)
stride = Z.shape[1]*Z.itemsize, Z.itemsize # ذخیره قدم‌های Z
print("Stride(as np.int16):",stride)
print("Z.stride(np.16):",Z.strides)
Z = np.arange(9).reshape(3,3).astype(np.int32)
stride= Z.shape[1]*Z.itemsize, Z.itemsize  # ذخیره قدم‌های Z
print("Stride(as np.int32):",stride)
print("Z.stride(np.32):",Z.strides)

Stride(as np.int16): (6, 2)
Z.stride(np.16): (6, 2)
Stride(as np.int32): (12, 4)
Z.stride(np.32): (12, 4)


<div dir="rtl" style="text-align: right;">


در این مثال، ما باید ۲ بایت (۱ مقدار) را برای حرکت به ستون بعدی پرش کنیم، اما ۶ بایت (۳ مقدار) را برای رسیدن به همان موقعیت در ردیف بعدی. بنابراین، قدم‌ها برای آرایه Z (6, 2) خواهد بود.
</p>
<p>با تمام این اطلاعات، ما می‌دانیم چگونه به یک مورد خاص دسترسی پیدا کنیم و به طور دقیق‌تر، چگونه افست‌های شروع و پایان را محاسبه کنیم:</p>
</div>

In [4]:
import numpy as np
Z = np.arange(9).reshape(3,3).astype(np.int16)
offset_start = 0
for i in range(Z.ndim):
  offset_start += Z.strides[i] * i 
  offset_end = offset_start + Z.itemsize 

print("Starting offset:", offset_start)
print("Ending offset:", offset_end)


Starting offset: 2
Ending offset: 4


<div dir="rtl" style="text-align: right;">
<p>بیایید صحت کارمان را با استفاده از روش تبدیل <a href="https://numpy.org/doc/stable/reference/generated/numpy.ndarray.tobytes.html">tobyte</a> بسنجیم. </p>
</div>


In [5]:
import numpy as np
Z = np.arange(9).reshape(3, 3).astype(np.int16)
index = 1, 1
print(Z[index].tobytes())
   #b'\x04\x00'
offset_start = 0
for i in range(Z.ndim):
  offset_start += Z.strides[i] * index[i]
  offset_end = offset_start + Z.itemsize
  
print(Z.tobytes()[offset_start:offset_end])#b'\x04\x00'

b'\x04\x00'
b'\x04\x00'


<div dir="rtl" style="text-align: right;">
<h2>چیدمان‌ها layouuts</h2>
<p>این آرایه را می‌توان از دیدگاه‌های مختلفی در نظر گرفت (چیدمان‌ها):</p>
<h3>چیدمان تکه‌ای</h3>
<p>چیدمان تکه‌ای را به عنوان یک ماتریس دو بعدی با x ردیف و y ستون در نظر بگیرید.</p>
<h4>مثال 1</h4>
<p>مثال زیر را در نظر بگیرید:</p>
<pre>
Z = np.arange(9).reshape(3, 3)
</pre>
<p>اینجا ۹ مقدار از ۰ تا ۸ گرفته شده و در قالب یک ماتریس دو بعدی با ابعاد (۳ * ۳) بازآرایی شده است.</p>
<p>Z چیدمان تکه‌ای زیر را دارد:</p>

</div>

<pre dir="ltr">
                  shape[1]
                    (=2)
               ┌───────────┐   

            ┌  ┌───┬╌╌╌┬───┐  ┐           
            │  │ 0 │   │ 2 │  │            ┌───┬───┐
            │  ├───┼╌╌╌┼───┤  │            │ 0 │ 2 │
   shape[0] │  ╎   ╎   ╎   ╎  │ len(V)  →  ├───┼───┤
    (=2)    │  ├───┼╌╌╌┼───┤  │  (=2)      │ 6 │ 8 │
            │  │ 6 │   │ 8 │  │            └───┴───┘
            └  └───┴╌╌╌┴───┘  ┘           
</pre>
<div dir="rtl" style="text-align: right;">
<p>در اینجا Z از ۹ مقدار از ۰ تا ۸ استفاده می‌کند و آنها را در یک ماتریس ۳ * ۳ مرتب می‌کند. V مقادیر را از گوشه‌های شبکه می‌گیرد، یعنی V دارای ۴ مقدار است.</p>
<p>V چیدمان تکه‌ای زیر را دارد:</p>
</div>


<pre dir="ltr">

                  shape[1]
                    (=2)
               ┌───────────┐   

            ┌  ┌───┬╌╌╌┬───┐  ┐           
            │  │ 0 │   │ 2 │  │            ┌───┬───┐
            │  ├───┼╌╌╌┼───┤  │            │ 0 │ 2 │
   shape[0] │  ╎   ╎   ╎   ╎  │ len(V)  →  ├───┼───┤
    (=2)    │  ├───┼╌╌╌┼───┤  │  (=2)      │ 6 │ 8 │
            │  │ 6 │   │ 8 │  │            └───┴───┘
            └  └───┴╌╌╌┴───┘  ┘           
</pre>

<div dir="rtl" style="text-align: right;">

<h3>چیدمان تکه‌ای صاف</h3>
<p>چیدمان تکه‌ای صاف را به عنوان یک ماتریس یک‌بعدی با یک ردیف و n ستون در نظر بگیرید.</p>
<h4>مثال 1</h4>
<p>مثال زیر را در نظر بگیرید:</p>
<pre>
Z = np.arange(9)
</pre>
<p>این ۹ شاخص را در حافظه کامپیوتر ایجاد می‌کند و مقادیر از ۰ تا ۸ را قرار می‌دهد.</p>
<p>Z چیدمان تکه‌ای صاف زیر را دارد:</p>

</div>

<pre>
   ┌───┬───┬───┬───┬───┬───┬───┬───┬───┐
   │ 0 │ 1 │ 2 │ 3 │ 4 │ 5 │ 6 │ 7 │ 8 │
   └───┴───┴───┴───┴───┴───┴───┴───┴───┘

   └───────────────────────────────────┘
                  Z.size
                   (=9)
</pre>


<div dir="rtl" style="text-align: right;">

<h4>مثال 2</h4>
<p>مثال زیر را در نظر بگیرید:</p>

</div>


<pre>
Z = np.arange(9).reshape(3,3).astype(np.int16)
V = Z[::2,::2]
V = V.reshape(1,4)
</pre>

<div dir="rtl" style="text-align: right;">
<p>در اینجا Z  ۹ مقدار از ۰ تا ۸ دارد و آنها را در یک ماتریس ۳ * ۳ مرتب می‌کند. V مقادیر را از گوشه‌های شبکه می‌گیرد، یعنی V دارای ۴ مقدار است.</p>
<p>V چیدمان تکه‌ای صاف زیر را دارد:</p>

</div>
<pre>
   ┌───┬╌╌╌┬───┬╌╌╌┬╌╌╌┬╌╌╌┬───┬╌╌╌┬───┐       ┌───┬───┬───┬───┐
   │ 0 │   │ 2 │   ╎   ╎   │ 6 │   │ 8 │   →   │ 0 │ 2 │ 6 │ 8 │
   └───┴╌╌╌┴───┴╌╌╌┴╌╌╌┴╌╌╌┴───┴╌╌╌┴───┘       └───┴───┴───┴───┘
   └─┬─┘   └─┬─┘           └─┬─┘   └─┬─┘
     └───┬───┘               └───┬───┘  
         └───────────┬───────────┘
                  V.size
                   (=4)
</pre>

<div dir="rtl" style="text-align: right;">
<h2>چیدمان‌های حافظه</h2>
<h3>چیدمان حافظه </h3>
<p>چیدمان حافظه را با ردیف‌هایی برابر با تعداد بایت‌ها و ستون‌هایی برابر با تعداد بایت‌ها تقسیم بر ۸ (یعنی <code>Z.itemsize</code>) در نظر بگیرید.</p>
<h4>مثال 1</h4>
<p>مثال زیر را در نظر بگیرید:</p>
</div>

<pre>
Z = np.arange(9).reshape(3, 3).astype(np.int16)
</pre>

<div dir="rtl" style="text-align: right;">
<p>در اینجا، تعداد ردیف‌ها ۱۶ و تعداد ستون‌ها ۲ است. تعداد کل بایت‌ها ۳ * ۳ * ۲ است که ۳ * ۳ اندازه شبکه است و هر سلول ۲ بایت می‌گیرد، پس مجموعاً ۱۸ بایت است. Z چیدمان حافظه زیر را دارد:</p>
</div>


<pre>

                            strides[1]
                              (=2)
                     ┌─────────────────────┐

             ┌       ┌──────────┬──────────┐ ┐
             │ p+00: │ 00000000 │ 00000000 │ │
             │       ├──────────┼──────────┤ │
             │ p+02: │ 00000000 │ 00000001 │ │ strides[0]
             │       ├──────────┼──────────┤ │   (=2x3)
             │ p+04  │ 00000000 │ 00000010 │ │
             │       ├──────────┼──────────┤ ┘
             │ p+06  │ 00000000 │ 00000011 │ 
             │       ├──────────┼──────────┤
   Z.nbytes  │ p+08: │ 00000000 │ 00000100 │
   (=3x3x2)  │       ├──────────┼──────────┤
             │ p+10: │ 00000000 │ 00000101 │
             │       ├──────────┼──────────┤
             │ p+12: │ 00000000 │ 00000110 │
             │       ├──────────┼──────────┤
             │ p+14: │ 00000000 │ 00000111 │
             │       ├──────────┼──────────┤
             │ p+16: │ 00000000 │ 00001000 │
             └       └──────────┴──────────┘

                     └─────────────────────┘   
                           Z.itemsize
                        Z.dtype.itemsize
                              (=2) 

</pre>




<div dir="rtl" style="text-align: right;">
<h4>مثال 2</h4>
<p>مثال زیر را در نظر بگیرید:</p>
<pre>
Z = np.arange(9).reshape(3,3).astype(np.int16)
V = Z[::2,::2]
</pre>
<p>در اینجا ما یک برش از Z را می‌گیریم، نتیجه نمایی از آرایه پایه Z است. در چیدمان حافظه زیر، از آنجا که آرایه فقط 4 مقدار دارد و هر مقدار 2 بایت است، بنابراین مجموع بایت‌ها 2 * 4 = 8 بایت است. V چیدمان حافظه زیر را دارد:</p>

</div>

<pre>

                 ┌        ┌──────────┬──────────┐ ┐             ┐
               ┌─┤  p+00: │ 00000000 │ 00000000 │ │             │
               │ └        ├──────────┼──────────┤ │ strides[1]  │
             ┌─┤    p+02: │          │          │ │   (=4)      │ 
             │ │ ┌        ├──────────┼──────────┤ ┘             │ 
             │ └─┤  p+04  │ 00000000 │ 00000010 │               │
             │   └        ├──────────┼──────────┤               │ strides[0] 
             │      p+06: │          │          │               │   (=12)
             │            ├──────────┼──────────┤               │
   V.nbytes ─┤      p+08: │          │          │               │
     (=8)    │            ├──────────┼──────────┤               │
             │      p+10: │          │          │               │
             │   ┌        ├──────────┼──────────┤               ┘              
             │ ┌─┤  p+12: │ 00000000 │ 00000110 │
             │ │ └        ├──────────┼──────────┤
             └─┤    p+14: │          │          │
               │ ┌        ├──────────┼──────────┤
               └─┤  p+16: │ 00000000 │ 00001000 │
                 └        └──────────┴──────────┘
                               
                          └─────────────────────┘
                                Z.itemsize
                             Z.dtype.itemsize
                                   (=2)

</pre>
